In [2]:
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta

In [82]:
ddf= pd.read_csv('2019-Nov.csv', chunksize=1000000)

for d in ddf:
    df = pd.DataFrame (d)
    break
df.head (10)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
5,2019-11-01 00:00:01 UTC,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
6,2019-11-01 00:00:01 UTC,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
7,2019-11-01 00:00:02 UTC,view,15900065,2053013558190408249,NaN,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
8,2019-11-01 00:00:02 UTC,view,12708937,2053013553559896355,NaN,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
9,2019-11-01 00:00:02 UTC,view,1004258,2053013555631882655,electronics.smartphone,apple,732.07,532647354,d2d3d2c6-631d-489e-9fb5-06f340b85be0


# Event Score

In [85]:
'''
We define the default event scores based on the event's rarity, 
  <There are various ways of dafining these scores based on the data>
'''
SCORES= 1/df.event_type.value_counts()*1e4
print(SCORES)

def EventScore(eventype, scores=SCORES):
  '''
  in this function we give a score for every event type 
  Alternative Approuch:
  this dataframe have three different event type and scores are like this
  cart=0.3
  view = 0.2
  purchase = 0.5
  '''
  return scores[eventype]

view        0.010339
purchase    0.561262
cart        0.668539
Name: event_type, dtype: float64


In [86]:
'''
adds a new column of event scores to the dataframe 
'''
df['event_score'] = df.event_type.apply(EventScore)

# Calling users by feature

In [211]:
'''
by calling "name_of_column" it returns which
customers are interested in the feature with what score
  <bigger score means that this costumer is more obsessed with the "name_of_column">

'''
def call_users_by(inp_column):
  user_df =df.groupby([inp_column,'user_id']).sum('event_score').sort_values('event_score',ascending=False)['event_score']
  # user_df.rename({'event_score':'Score'})
  return pd.DataFrame(user_df).rename(columns = {'event_score':'Score'})

'''
cat_user_score =df.groupby(['product_id','user_id']).sum('event_score').sort_values('event_score',ascending=False)['event_score']
pd.DataFrame(cat_user_score)
call_users_by('product_id').loc[1005217]  #example
'''

"\ncat_user_score =df.groupby(['product_id','user_id']).sum('event_score').sort_values('event_score',ascending=False)['event_score']\npd.DataFrame(cat_user_score)\ncall_users_by('product_id').loc[1005217]  #example\n"

#### Example:

In [217]:
call_users_by('brand').loc['xiaomi']  #example

,Score
user_id,
521350556,42.196753
530131574,13.576223
515524354,9.962157
513022593,8.318401
566369023,8.063818
...,...
512774095,0.010339
512770864,0.010339
512772509,0.010339


# Product info

In [214]:
'''
based on need to product data, including product_id, brand or category_id

'''
def product_info(product_id= None, brand=None, category_id=None):
  if product_id is not None:
    out= df[['product_id', 'category_id', 'category_code', 'brand', 'price']].drop_duplicates(keep='first').set_index('product_id')#.value_counts()
    return out.loc[product_id].iloc[0]
  elif brand is not None:
    out= df[['product_id', 'category_id', 'category_code', 'brand', 'price']].drop_duplicates(keep='first').set_index('brand')#.value_counts()
    return out.loc[brand]
  elif category_id is not None:
    out= df[['product_id', 'category_id', 'category_code', 'brand', 'price']].drop_duplicates(keep='first').set_index('category_id')#.value_counts()
    return out.loc[category_id]

#### Example

In [215]:
product_info(brand='xiaomi')

,product_id,category_id,category_code,price
brand,,,,
xiaomi,1003461,2053013555631882655,electronics.smartphone,489.07
xiaomi,1004775,2053013555631882655,electronics.smartphone,183.27
xiaomi,5100503,2053013553375346967,NaN,22.68
xiaomi,1005161,2053013555631882655,electronics.smartphone,211.92
xiaomi,1004877,2053013555631882655,electronics.smartphone,84.65
...,...,...,...,...
xiaomi,4802720,2053013554658804075,electronics.audio.headphone,16.19
xiaomi,18300433,2053013558945383017,accessories.bag,49.94
xiaomi,18300978,2053013558945383017,accessories.bag,24.17
